In [1]:
import pandas as pd
sam_df_path = 'C:/Users/sjmal/OneDrive/Desktop/ML/2021/mlchallenge_set_2021_edited.txt'
sam_valid_path
df = pd.read_table()
valid = pd.read_table('C:/Users/sjmal/OneDrive/Desktop/ML/2021/mlchallenge_set_validation.tsv',sep='\t')
df.columns=['category','primary_image_url','All Links','Tags','ID']
valid.columns=['ID', 'Group']

In [2]:
#create dictionary of groups with list of IDs in that group
dict = {}
total=0
for index,row in valid.iterrows():
    if row['Group'] in dict:
        dict[row['Group']].append(row['ID'])
    else:
        dict[row['Group']] = [row['ID']]
#list of groups with multiple IDs
matches = []
for key,value in dict.items():
    if len(value) > 1:
        total+=len(value)
        matches.append(key)
print(len(matches))
print(total)

990
2480


In [3]:
print(df.head())
print(valid.head())

   Category                                          Main Link  \
0         2  https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/iYYA...   
1         2  https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPIA...   
2         2  https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/flIA...   
3         2  http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/XHcAA...   
4         2  https://i.ebayimg.com/00/s/MTA2N1gxNjAw/z/scsA...   

                                           All Links  \
0  https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/iYYA...   
1  https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPIA...   
2  https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/flIA...   
3  http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/XHcAA...   
4  https://i.ebayimg.com/00/s/MTA2N1gxNjAw/z/scsA...   

                                                Tags  ID  
0  (Brand:Shimano,US Shoe Size (Men's):4.5,Modifi...   0  
1  (Color:Gray/White,Country/Region of Manufactur...   1  
2  (Style:Cleats,Color:White Orange,US Shoe Size ...   2  
3  (Width:Medium (D, M),US Siz

In [3]:
#split links into list (for now only does first 100)
i = 0
for link in df['All Links'][0:100]:
    df['All Links'][i] = link.split(';')
    i+=1
print(df['All Links'][0:100])

<ipython-input-3-5e0f35be6386>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['All Links'][i] = link.split(';')


0     [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/iYY...
1     [https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPI...
2     [https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/flI...
3     [http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/XHcA...
4     [https://i.ebayimg.com/00/s/MTA2N1gxNjAw/z/scs...
                            ...                        
95    [https://i.ebayimg.com/00/s/MTU5OVgxNTgx/z/vMA...
96    [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/GoA...
97    [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/Hm0...
98    [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/VhA...
99    [https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/n4o...
Name: All Links, Length: 100, dtype: object


In [42]:
import re
def register_attributes(attribute, all_attributes):
    attribute = re.sub(r'[()]','', attribute)
    attribute = re.split(r',', attribute)
    attribute = [a.split(':') for a in attribute]
    for i, a in enumerate(attribute):
        attribute[i] = [s.strip() for s in a]
        all_attributes.add(attribute[i][0])
    #print(f'atttribute is: {attribute}')
    mapping = {}
    #for i in range(len(attribute) - 1):
    #    if i == len(attribute) - 2:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:]
    #    else:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:-1]
    return(attribute)

def map_attributes(attribute, num_attributes, index_to_attr):
    attribute = re.sub(r'[()]','', attribute)
    attribute = re.split(r',', attribute)
    attribute = [a.split(':') for a in attribute]
    all_attributes_for_row = [None] * num_attributes
    for i, a in enumerate(attribute):
        attribute[i] = [s.strip() for s in a]
        #print(f'index: {attr_to_index[attribute[i][0]]}')
        if len(attribute[i]) > 1:
            all_attributes_for_row[attr_to_index[attribute[i][0]]] = attribute[i][1]
    mapping = {}
    #for i in range(len(attribute) - 1):
    #    if i == len(attribute) - 2:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:]
    #    else:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:-1]
    return all_attributes_for_row

all_attributes = set()
all_maps = []
for index,row in df[0:5000].iterrows():
    register_attributes(row['Tags'], all_attributes)

all_attributes = list(all_attributes)
attr_to_index = {all_attributes[i]: i for i in range(len(all_attributes))}
#print(attr_to_index)
#print(f'numAttributes: {len(all_attributes)}')

for index,row in df[0:5000].iterrows():
    all_maps.append(map_attributes(row['Tags'], len(all_attributes), attr_to_index))
#print(all_maps)


In [43]:
#dataframe with attribute values
categories = pd.DataFrame(all_maps)
#print(categories)

In [44]:
from sklearn.preprocessing import OneHotEncoder
x = OneHotEncoder().fit_transform(categories)
print(x)
print(type(x))

MemoryError: Unable to allocate 11.6 MiB for an array with shape (3035000,) and data type int32

In [36]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(compute_distances=True,compute_full_tree = True,distance_threshold = .01,n_clusters=None).fit(x.todense())
print(clustering)

AgglomerativeClustering(compute_distances=True, compute_full_tree=True,
                        distance_threshold=0.01, n_clusters=None)


In [37]:
print(clustering.labels_)
print(clustering.distances_)
print(clustering.n_connected_components_)

[95 61 55 51 62 64 78  0 91 83 79 87 50 76 70 52 98 97 86 93 81 85 90 56
 67 80 66 96 65 84 49 42 47 27 74  0 57 32 94 92 63 45 68 89 46 88 39 24
 59 22 43 75 53 72 30 60 77 41 82 29 71 54 25 73 69 58 33 20 48 40 23 38
 37 44 26 14 35 34 19 21 28 10  9 31 18 36  4 15 16 17  8 13  7  3 12  6
 11  5  1  2]
[0.         1.41421356 1.41421356 1.41421356 1.41421356 1.41421356
 1.41421356 1.41421356 1.41421356 1.41421356 1.82574186 1.91485422
 2.         2.         2.         2.         2.         2.
 2.         2.         2.         2.         2.         2.
 2.1602469  2.1602469  2.1602469  2.23606798 2.28035085 2.30940108
 2.30940108 2.30940108 2.30940108 2.30940108 2.4377976  2.44948974
 2.44948974 2.44948974 2.44948974 2.44948974 2.44948974 2.44948974
 2.44948974 2.5819889  2.5819889  2.5819889  2.7080128  2.7080128
 2.7080128  2.7080128  2.76887462 2.82842712 2.82842712 2.82842712
 2.82842712 2.82842712 3.         3.         3.05505046 3.05505046
 3.16227766 3.26598632 3.32665999 3.33809

In [38]:
%matplotlib qt
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram
import numpy as np
from PIL import Image
import requests
from io import BytesIO

def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    #response = requests.get(url)
    #img = Image.open(BytesIO(response.content))
    link_labels = [df['Tags'][i] for i in clustering.labels_]
    dendrogram(linkage_matrix, labels = link_labels)
plot_dendrogram(clustering)
plt.set_urls(df['Main Link'][i] for i in clustering.labels_)
plt.show()

AttributeError: module 'matplotlib.pyplot' has no attribute 'set_urls'